In [1]:
import pandas as pd
import pickle

file_path = '/home/uework/AiWeb/pwz/github/data/dataset/dialogue-lua/qa-cn-2.dat'
with open(file_path, 'rb') as f:
    examples = pickle.load(f)


# Initialize lists to store prompts and responses
prompts = []
responses = []

# Parse out prompts and responses from examples
for example in examples:
  try:
    split_example = example.split('-----------')
    if split_example[1].strip() and split_example[3].strip():
        prompts.append(split_example[1].strip())
        responses.append(split_example[3].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
    'response': responses
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.to_csv('data_cn.csv',index=False)
df.tail()



There are 321 successfully-generated examples. Here are the first few:


,prompt,response
354,如何获取行会成员列表？,```lua\n-- 获取行会成员列表\nfunction get_member_ids(g...
355,如何获取行会成员对象？,```lua\n-- 获取行会成员对象\nfunction get_member_info(...
356,如何设置lua拓展数据？,```lua\n-- 设置lua拓展数据\nfunction set_lua_ext(gui...
357,如何获取lua拓展数据？,```lua\n-- 获取lua拓展数据\nfunction get_lua_ext(gui...
358,如何广播给公会成员？,```lua\n-- 广播给公会成员\nfunction broadcast(guild_o...


In [14]:
from datasets import load_dataset

dataset = load_dataset('csv', data_files={'data_cn.csv'})
dataset = dataset['train']
dataset = dataset.add_column("completion",dataset['response'])
dataset = dataset.train_test_split(0.05)
dataset.save_to_disk('/home/uework/AiWeb/pwz/github/data/dataset/dialogue-lua')
dataset

Found cached dataset csv (/home/uework/.cache/huggingface/datasets/csv/default-a30dc41485662cf5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 349.55it/s]


DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'completion'],
        num_rows: 303
    })
    test: Dataset({
        features: ['prompt', 'response', 'completion'],
        num_rows: 16
    })
})

In [5]:
import pickle
import pandas as pd
file_path = '/home/uework/AiWeb/pwz/github/data/prompt/q-cn-test.dat'
with open(file_path, 'rb') as f:
    examples = pickle.load(f)


# Initialize lists to store prompts and responses
prompts = []

# Parse out prompts and responses from examples
for example in examples:
  try:
    split_example = example.split('-----------')
    if split_example[1].strip():
        prompts.append(split_example[1].strip())
  except:
    pass

# Create a DataFrame
df = pd.DataFrame({
    'prompt': prompts,
})

# Remove duplicates
df = df.drop_duplicates()

print('There are ' + str(len(df)) + ' successfully-generated examples. Here are the first few:')

df.to_csv('/home/uework/AiWeb/pwz/github/data/prompt/test_quary.csv',index=False)
df.tail()


There are 171 successfully-generated examples. Here are the first few:


,prompt
166,玩家放技能时我需要触发一些事件，有没有相关的API函数？
167,在玩家准备释放技能前我想执行一些操作，应该用哪个API函数？
168,玩家切换自动战斗模式时我需要做一些操作，有没有对应的API函数？
169,我想在玩家附加buff时进行一些操作，应该用哪个API函数？
170,当卸下buff时我想执行一些操作，我应该用哪个API函数？


/home/uework/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from utils import dialogue_to_json
from tqdm import tqdm
from datasets import DatasetDict
from datasets import load_from_disk
openassistant = load_from_disk('/home/uework/AiWeb/pwz/github/data/dataset/openassistant')
train_json_list = []
# train data
for row in tqdm(openassistant['train']['text']):
    # Convert to JSON format
    json_data = dialogue_to_json(row)
    # Append to list
    train_json_list.append(json_data['messages'])
test_json_list = []
# test data
for row in tqdm(openassistant['test']['text']):
    # Convert to JSON format
    json_data = dialogue_to_json(row)
    # Append to list
    test_json_list.append(json_data['messages'])
# create dataset
train_dataset = openassistant['train'].add_column('messages', train_json_list)
test_dataset = openassistant['test'].add_column('messages', test_json_list)
# save dataset
openassistant_json = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})
openassistant_json.save_to_disk('/home/uework/AiWeb/pwz/github/data/dataset/openassistant_json')
openassistant_json
    

Saving the dataset (1/1 shards): 100%|██████████| 518/518 [00:00<00:00, 107455.83 examples/s]
